In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from Color import *

# Matplotlib font configuration
from MatplotlibFonts import *
from BenchmarkConfiguration import *

# Change the font size
rcParams.update({"font.size": 11})

In [3]:
RESULT_DIR='../benchmarks/results-powertrace'

In [4]:
# Benchmarks in order
Benchmarks = [
    'coremark',
    'sha',
    'crc',
    'aes',
    'dijkstra',
    'picojpeg'
]

In [5]:
df = {}

Names=['on-time', 'stdev', 'cycles', 'power-failures']
def load_benchmark(bench):
    file = RESULT_DIR + '/' + bench + '/powertrace-stats.csv'
    print('Loading file: ' + file)
    return pd.read_csv(file, names=Names)

for bench in Benchmarks:
    df[bench] = load_benchmark(bench).groupby('on-time').mean()
    df[bench]['exec'] = df[bench]['cycles'] / df[bench].loc[0]['cycles']
    df[bench] = df[bench].reset_index()

     

Loading file: ../benchmarks/results-powertrace/coremark/powertrace-stats.csv
Loading file: ../benchmarks/results-powertrace/sha/powertrace-stats.csv
Loading file: ../benchmarks/results-powertrace/crc/powertrace-stats.csv
Loading file: ../benchmarks/results-powertrace/aes/powertrace-stats.csv
Loading file: ../benchmarks/results-powertrace/dijkstra/powertrace-stats.csv
Loading file: ../benchmarks/results-powertrace/picojpeg/powertrace-stats.csv


In [6]:
df['coremark']

,on-time,stdev,cycles,power-failures,exec
0,0,1000.0,6270699.0,0.0,1.000000
1,50000,1000.0,6295037.0,125.0,1.003881
2,100000,1000.0,6294592.0,62.0,1.003810
3,200000,1000.0,6274351.0,31.0,1.000582
4,500000,1000.0,6272258.0,12.0,1.000249
5,1000000,1000.0,6272027.0,6.0,1.000212
6,2000000,1000.0,6271616.0,3.0,1.000146
7,5000000,1000.0,6270798.0,1.0,1.000016
8,10000000,1000.0,6270699.0,0.0,1.000000
9,20000000,1000.0,6270699.0,0.0,1.000000


In [8]:
# Create the latex table holding all the info
OnTimes = [
#    0, # Cont
    50000000,
    10000000,
    5000000,
    1000000,
    500000,
    100000,
    50000,
]
OnTimes.reverse()

tex=''
def tex_addl(t):
    global tex
    tex += t + '\n'
    
def tex_add(t):
    global tex
    tex += t
    
def colorbox(exec_val, t):
    perc = exec_val*6;
    if round(exec_val,2) > 0.0001:
        perc += 15
        
    return r'\colorbox{orange!'+str(int(perc))+r'!white}{'+t+r'}'
    
# Add a group to change the table spacing
tex_addl(r'% table automatically generated by PowertracePlot.ipynb')
#tex_addl(r'\begingroup')
#tex_addl(r'\setlength{\tabcolsep}{4pt} % Default value: 6pt')

# Begin the table
#tex_addl(r'\begin{table*}[]')
#tex_addl(r'\centering')
#tex_addl(r'\small')
tex_addl(r'\begin{tabular}{@{}llcrcrcrcrcrcr@{}}')
tex_addl(r'\toprule')

# Add the benchmark headers
tex_add(r'\multicolumn{2}{l}{power \emph{on} duration}')
for bench in Benchmarks:
    tex_add(r' & \multicolumn{2}{c}{\textbf{\textsf{'+BenchmarkNameMap[bench]+'}}}')
tex_addl(r' \\ \midrule')

# Add the power-fail header
tex_add(r'\textbf{clock cycles} & \multicolumn{1}{l|}{\textbf{time at \{8\,MHz, 50\,MHz\}}}')
for bench in Benchmarks:
    tex_add(r' & \textbf{$\mathcal{O}$} & \textbf{$\mathcal{P}$}')
tex_addl(r' \\')

# Add the actual numbers
for ontime in OnTimes:
    if ontime == 0:
        tex_add(r'\textit{continuous}')
        tex_add(r' & ')
        tex_add(r' & \multicolumn{1}{l|}{} ')
    else:
        tex_add('$'+str(ontime)+'$')
        on_8MHz = ontime/8000
        if on_8MHz >= 100.0:
            on_8MHz = '${:.0f}'.format(ontime/8000) + r'\,$ms'
        else:
            on_8MHz = '${:.1f}'.format(ontime/8000) + r'\,$ms'
        on_50MHz = '${:.0f}'.format(ontime/50000) + r'\,$ms'
        #tex_add(r' & '+on_8MHz)
        tex_add(r' & \multicolumn{1}{l|}{\{'+on_8MHz+', '+on_50MHz+'\} } ')
            
    for bench in Benchmarks:
        exec_val = df[bench].set_index('on-time').loc[ontime]['exec']
        pf_val = df[bench].set_index('on-time').loc[ontime]['power-failures']
        
        exec_val = (exec_val-1)*100
        
        exec_str = '{:.2f}'.format(exec_val)
        tex_add(r' & '+colorbox(exec_val,exec_str+r'\%'))
        tex_add(r' & '+str(int(pf_val)))
    tex_addl(r' \\')
tex_addl(r' \bottomrule')

# End the table
tex_addl(r'\end{tabular}')
#tex_addl(r'\end{table*}')

# End the group
#tex_addl(r'\endgroup')

print(tex)

# Save the table
with open('plots/powertrace-table.tex', "w") as f:
    # Writing data to a file
    f.write(tex)

% table automatically generated by PowertracePlot.ipynb
\begin{tabular}{@{}llcrcrcrcrcrcr@{}}
\toprule
\multicolumn{2}{l}{power \emph{on} duration} & \multicolumn{2}{c}{\textbf{\textsf{CoreMark}}} & \multicolumn{2}{c}{\textbf{\textsf{SHA}}} & \multicolumn{2}{c}{\textbf{\textsf{CRC}}} & \multicolumn{2}{c}{\textbf{\textsf{Tiny AES}}} & \multicolumn{2}{c}{\textbf{\textsf{Dijkstra}}} & \multicolumn{2}{c}{\textbf{\textsf{picojpeg}}} \\ \midrule
\textbf{clock cycles} & \multicolumn{1}{l|}{\textbf{time at \{8\,MHz, 50\,MHz\}}} & \textbf{$\mathcal{O}$} & \textbf{$\mathcal{P}$} & \textbf{$\mathcal{O}$} & \textbf{$\mathcal{P}$} & \textbf{$\mathcal{O}$} & \textbf{$\mathcal{P}$} & \textbf{$\mathcal{O}$} & \textbf{$\mathcal{P}$} & \textbf{$\mathcal{O}$} & \textbf{$\mathcal{P}$} & \textbf{$\mathcal{O}$} & \textbf{$\mathcal{P}$} \\
$50000$ & \multicolumn{1}{l|}{\{$6.2\,$ms, $1\,$ms\} }  & \colorbox{orange!17!white}{0.39\%} & 125 & \colorbox{orange!32!white}{2.87\%} & 380 & \colorbox{orange!58!white}{